# EDM-IDM conversion

This notebook converts Europeana creator and/or contributor information to IDM


In [3]:
import os
from pathlib import Path
from SPARQLWrapper import SPARQLWrapper, JSON, XML, TURTLE, N3, RDF, RDFXML, CSV, TSV, JSONLD, DIGEST
import requests
from urllib import parse, error
from rdflib import Graph, RDF, URIRef, Literal, BNode, Namespace
import warnings
import time


# Modualised version

In [4]:
contributorquery="""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX edm: <http://www.europeana.eu/schemas/edm/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX p: <http://www.wikidata.org/prop/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX bd: <http://www.bigdata.com/rdf#>
PREFIX idm: <https://www.intavia.org/idm/>
PREFIX idmrole: <https://www.intavia.org/idm/role/>
PREFIX bioc: <http://ldf.fi/schema/bioc/>
PREFIX psv: <http://www.wikidata.org/prop/statement/value/>
PREFIX schema: <http://schema.org/>
PREFIX psn: <http://www.wikidata.org/prop/statement/value-normalized/>
PREFIX wds: <http://www.wikidata.org/entity/statement/>
PREFIX pq: <http://www.wikidata.org/prop/qualifier/>

CONSTRUCT {
	?cho a crm:E24_Physical_Human_Made_Thing,
		idm:CHO_Proxy;
		crm:P1_is_identified_by ?cho_1476Title;
		bioc:bearer_of ?choProductionEventRole.
	?cho rdfs:label ?dc_title .

	?edm_object crm:P70_documents ?cho;
		a crm:E31_Document.
	?cho_1476Title rdfs:label ?dc_title;
		a crm:E35_Title .

	?choProductionEventRole a bioc:Thing_Role .
	?choProductionEvent bioc:occured_in_the_presence_of_in_role ?choProductionEventRole ;
		 a crm:E12_Production ;
		 crm:P4_has_time-span ?choProductionTimespan ;
		 rdfs:label ?eventLabel .

	?choProductionTimespan rdfs:label ?dc_date;
		crm:P82a_beginning_of_the_begin ?dc_date ;
		crm:P82b_end_of_the_end ?dc_date .

	?choProductionEvent crm:P7_took_place_at ?locationofcreation.
	?locationofcreation a crm:E53_Place .
	?choProductionEvent bioc:had_participant_in_role ?producingArtistRole .
	?artist bioc:bearer_of ?producingArtistRole .
	?producingArtistRole a idmrole:contributing_artist .
	idmrole:contributing_artist rdfs:label "contributing artist"@en
}
WHERE 
{  		
	?dc_creator_uri owl:sameAs ?artistUri .
	?cho dc:contributor ?dc_creator_uri .

	BIND(REPLACE(STR(?cho), "http://", "https://www.intavia.eu/productionthingrole/") as ?choProductionEventRoleStr)      
	BIND(IRI(?choProductionEventRoleStr) as ?choProductionEventRole)
	BIND(REPLACE(STR(?cho), "http://", "https://www.intavia.eu/production_event/") as ?choProductionEventStr)      
	BIND(IRI(?choProductionEventStr) as ?choProductionEvent)
	BIND(IRI(REPLACE(STR(?artistUri), "http://", STR(idmrole:producing_artist))) as ?producingArtistRole)
	BIND(REPLACE(STR(?cho), "http://", "https://www.intavia.org/chotitle/") as ?cho_TitleStr)
	BIND(IRI(?cho_TitleStr) as ?cho_1476Title)
	OPTIONAL {?cho dc:coverage ?dc_coverage_uri}
	OPTIONAL {?cho dc:type ?dc_type_uri }

	# Shortcut technique to avoid hopping many nodes (for the 2 subqueries below and )
	BIND(IRI(REPLACE(str(?cho), "http://data.europeana.eu/proxy/europeana/", "http://data.europeana.eu/proxy/provider/")) AS ?provider_proxy)
	BIND(IRI(REPLACE(str(?cho), "http://data.europeana.eu/proxy/europeana/", "http://data.europeana.eu/aggregation/provider/")) AS ?provider_agg)
	BIND(IRI(REPLACE(str(?cho), "http://data.europeana.eu/proxy/europeana/", "http://data.europeana.eu/item/")) AS ?item)

	?provider_proxy edm:type ?edm_type
	OPTIONAL {?provider_proxy dc:contributor ?dc_creator}
	OPTIONAL {?provider_proxy dc:coverage ?dc_coverage.
		BIND(REPLACE(STR(?cho), "http://", "https://www.intavia.eu/location_of_production/") as ?locationofcreation)      
	}
	OPTIONAL {?provider_proxy dc:date ?dc_date}
	OPTIONAL {?provider_proxy dc:title ?dc_title}
	OPTIONAL {?provider_agg edm:object ?edm_object}


}

"""

creatorquery="""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX edm: <http://www.europeana.eu/schemas/edm/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX p: <http://www.wikidata.org/prop/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX bd: <http://www.bigdata.com/rdf#>
PREFIX idm: <https://www.intavia.org/idm/>
PREFIX idmrole: <https://www.intavia.org/idm/role/>
PREFIX bioc: <http://ldf.fi/schema/bioc/>
PREFIX psv: <http://www.wikidata.org/prop/statement/value/>
PREFIX schema: <http://schema.org/>
PREFIX psn: <http://www.wikidata.org/prop/statement/value-normalized/>
PREFIX wds: <http://www.wikidata.org/entity/statement/>
PREFIX pq: <http://www.wikidata.org/prop/qualifier/>

CONSTRUCT {
	?cho a crm:E24_Physical_Human_Made_Thing,
		idm:CHO_Proxy;
		crm:P1_is_identified_by ?cho_1476Title;
		bioc:bearer_of ?choProductionEventRole.
	?cho rdfs:label ?dc_title .

	?edm_object crm:P70_documents ?cho;
		a crm:E31_Document.
	?cho_1476Title rdfs:label ?dc_title;
		a crm:E35_Title .

	?choProductionEventRole a bioc:Thing_Role .
	?choProductionEvent bioc:occured_in_the_presence_of_in_role ?choProductionEventRole ;
		 a crm:E12_Production ;
		 crm:P4_has_time-span ?choProductionTimespan ;
		 rdfs:label ?eventLabel .

	?choProductionTimespan rdfs:label ?dc_date;
		crm:P82a_beginning_of_the_begin ?dc_date ;
		crm:P82b_end_of_the_end ?dc_date .

	?choProductionEvent crm:P7_took_place_at ?locationofcreation.
	?locationofcreation a crm:E53_Place .
	?choProductionEvent bioc:had_participant_in_role ?producingArtistRole .
	?artist bioc:bearer_of ?producingArtistRole .
	?producingArtistRole a idmrole:producing_artist .
	idmrole:producing_artist rdfs:label "producing artist"@en
}
WHERE 
{  		
	?dc_creator_uri owl:sameAs ?artistUri .
	?cho dc:creator ?dc_creator_uri .

	BIND(REPLACE(STR(?cho), "http://", "https://www.intavia.eu/productionthingrole/") as ?choProductionEventRoleStr)      
	BIND(IRI(?choProductionEventRoleStr) as ?choProductionEventRole)
	BIND(REPLACE(STR(?cho), "http://", "https://www.intavia.eu/production_event/") as ?choProductionEventStr)      
	BIND(IRI(?choProductionEventStr) as ?choProductionEvent)
	BIND(IRI(REPLACE(STR(?artistUri), "http://", STR(idmrole:producing_artist))) as ?producingArtistRole)
	BIND(REPLACE(STR(?cho), "http://", "https://www.intavia.org/chotitle/") as ?cho_TitleStr)
	BIND(IRI(?cho_TitleStr) as ?cho_1476Title)
	OPTIONAL {?cho dc:coverage ?dc_coverage_uri}
	OPTIONAL {?cho dc:type ?dc_type_uri }

	# Shortcut technique to avoid hopping many nodes (for the 2 subqueries below and )
	BIND(IRI(REPLACE(str(?cho), "http://data.europeana.eu/proxy/europeana/", "http://data.europeana.eu/proxy/provider/")) AS ?provider_proxy)
	BIND(IRI(REPLACE(str(?cho), "http://data.europeana.eu/proxy/europeana/", "http://data.europeana.eu/aggregation/provider/")) AS ?provider_agg)
	BIND(IRI(REPLACE(str(?cho), "http://data.europeana.eu/proxy/europeana/", "http://data.europeana.eu/item/")) AS ?item)

	?provider_proxy edm:type ?edm_type
	OPTIONAL {?provider_proxy dc:creator ?dc_creator}
	OPTIONAL {?provider_proxy dc:coverage ?dc_coverage.
		BIND(REPLACE(STR(?cho), "http://", "https://www.intavia.eu/location_of_production/") as ?locationofcreation)      
	}
	OPTIONAL {?provider_proxy dc:date ?dc_date}
	OPTIONAL {?provider_proxy dc:title ?dc_title}
	OPTIONAL {?provider_agg edm:object ?edm_object}


}

"""

### Iterate over files in a folder

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:

# Load InTaVia data which contains persons with Wikidata URI
i_file = 'InTaVia_all_persons_with_WikidataURI_query_results.ttl'
#name of the folder + filename
dirstr = 'C:/Users/vbr240/git/InTaVia2/InTaVia_EuropeanaCHO_files/test/Contributor/OnlyInTaVia_at_nl_fi_si_URI_list_df_WikidataContributor_' 
#starting folder number
nr = 10000
#last file nr
maxnr = 270000

while nr < maxnr:
    print(str(nr))
    # Load Europeana data from a specific folder
    dir_path = dirstr+str(nr)+'.csv/'
    arr = os.listdir(dir_path)
    outputfile = 'result_contributor_'+str(nr)+'.ttl'
    format = 'ttl'
    g2 = Graph()

    i = 0
    query_content = contributorquery

    tic = time.perf_counter()

    for onefile in arr:
        # some arbitrary stop condition
        if i<99999999:
            g = Graph()
            g.parse(i_file) # load wikidata
            #print(onefile)
            with open(dir_path+onefile, 'r', encoding="utf8") as file:
                stringdata = file.read().replace('^^<http://www.w3.org/2001/XMLSchema#hexBinary>', '')
            g.parse(data=stringdata)
            result = g.query(query_content)
            g2+=result
            print(str(i)+'..', end='')
            i+=1

    toc = time.perf_counter()
    print(f"done in {toc - tic:0.4f} seconds")

          #  g.parse(dir_path+onefile, format="ttl")

    try:
        g2.serialize(destination=outputfile)
    except:
        print('some error occurred')
    nr += 10000


10000
0..1..2..3..4..5..6..7..8..9..10..11..12..13..14..15..16..17..18..19..20..21..22..23..24..25..26..27..28..29..30..31..32..33..34..35..36..37..38..39..40..41..42..43..44..45..46..47..48..49..50..51..52..53..54..55..56..57..58..59..60..61..62..63..64..65..66..67..68..69..70..71..72..73..74..75..76..77..78..79..80..81..82..83..84..85..86..87..88..89..90..91..92..93..94..95..96..97..98..99..100..101..102..103..104..105..106..107..108..109..110..111..112..113..114..115..116..117..118..119..120..121..122..123..124..125..126..127..128..129..130..131..132..133..134..135..136..137..138..139..140..141..142..143..144..145..146..147..148..149..150..151..152..153..154..155..156..157..158..159..160..161..162..163..164..165..166..167..168..169..170..171..172..173..174..175..176..177..178..179..180..181..182..183..184..185..186..187..188..189..190..191..192..193..194..195..196..197..198..199..200..201..202..203..204..205..206..207..208..209..210..211..212..213..214..215..216..217..218..219..220.

http://iconclass.org/82A(GODFREY OF BOUILLON) does not look like a valid URI, trying to serialize this will break.
https://www.intavia.org/idm/role/producing_artisticonclass.org/82A(GODFREY OF BOUILLON) does not look like a valid URI, trying to serialize this will break.


1267..1268..1269..1270..1271..1272..1273..1274..1275..1276..1277..1278..1279..1280..1281..1282..1283..1284..1285..1286..1287..1288..1289..1290..1291..1292..1293..1294..1295..1296..1297..1298..1299..1300..1301..1302..1303..1304..1305..1306..1307..1308..1309..1310..1311..1312..1313..1314..1315..1316..1317..1318..1319..1320..1321..1322..1323..1324..1325..1326..1327..1328..1329..1330..1331..1332..1333..1334..1335..1336..1337..1338..1339..1340..1341..1342..1343..1344..1345..1346..1347..1348..1349..1350..1351..1352..1353..1354..1355..1356..1357..1358..1359..1360..1361..1362..1363..1364..1365..1366..1367..1368..1369..1370..1371..1372..1373..1374..1375..1376..1377..1378..1379..1380..1381..1382..1383..1384..1385..1386..1387..1388..1389..1390..1391..1392..1393..1394..1395..1396..1397..1398..1399..1400..1401..1402..1403..1404..1405..1406..1407..1408..1409..1410..1411..1412..1413..1414..1415..1416..1417..1418..1419..1420..1421..1422..1423..1424..1425..1426..1427..1428..1429..1430..1431..1432..1433

 http://datos.bne.es/resource/XX948112 does not look like a valid URI, trying to serialize this will break.
 http://datos.bne.es/resource/XX1689854 does not look like a valid URI, trying to serialize this will break.
 http://datos.bne.es/resource/XX874892 does not look like a valid URI, trying to serialize this will break.


1573..1574..

...

1315..1316..1317..1318..1319..1320..1321..1322..1323..1324..1325..1326..1327..1328..1329..1330..1331..1332..1333..1334..1335..1336..1337..1338..1339..1340..1341..1342..1343..1344..1345..1346..1347..1348..1349..1350..1351..1352..1353..1354..1355..1356..1357..1358..1359..1360..1361..1362..1363..done in 3834.9508 seconds
50000
0..1..2..3..4..5..6..7..8..9..10..11..12..13..14..15..16..17..18..19..20..21..22..23..24..25..26..27..28..29..30..31..32..33..34..35..36..37..38..39..40..41..42..43..44..45..46..47..48..49..50..51..52..53..54..55..56..57..58..59..60..61..62..63..64..65..66..67..68..69..70..71..72..73..74..75..76..77..78..79..80..81..82..83..84..85..86..87..88..89..90..91..92..93..94..95..96..97..98..99..100..101..102..103..104..105..106..107..108..109..110..111..112..113..114..115..116..117..118..119..120..121..122..123..124..125..126..127..128..129..130..131..132..133..134..135..136..137..138..139..140..141..142..143..144..145..146..147..148..149..150..151..152..153..154..155..156.

 http://datos.bne.es/resource/XX1933670 does not look like a valid URI, trying to serialize this will break.
 http://datos.bne.es/resource/XX1486593 does not look like a valid URI, trying to serialize this will break.
 http://datos.bne.es/resource/XX843538 does not look like a valid URI, trying to serialize this will break.
 http://datos.bne.es/resource/XX1204378 does not look like a valid URI, trying to serialize this will break.
 http://datos.bne.es/resource/XX2901779 does not look like a valid URI, trying to serialize this will break.


615..

 http://datos.bne.es/resource/XX1521375 does not look like a valid URI, trying to serialize this will break.
 http://datos.bne.es/resource/XX1076823 does not look like a valid URI, trying to serialize this will break.
 http://datos.bne.es/resource/XX1517303 does not look like a valid URI, trying to serialize this will break.
 http://datos.bne.es/resource/XX1295030 does not look like a valid URI, trying to serialize this will break.
 http://datos.bne.es/resource/XX1151631 does not look like a valid URI, trying to serialize this will break.
 http://datos.bne.es/resource/XX1517351 does not look like a valid URI, trying to serialize this will break.
 http://datos.bne.es/resource/XX1225886 does not look like a valid URI, trying to serialize this will break.
 http://datos.bne.es/resource/XX1225885 does not look like a valid URI, trying to serialize this will break.


616..617..618..619..620..621..622..623..624..625..626..627..628..629..630..631..632..633..634..635..636..637..638..639..640..641..642..643..644..645..646..647..648..649..650..651..652..653..654..655..656..657..658..659..660..661..662..663..664..665..666..667..668..669..670..671..672..673..674..675..676..677..678..679..680..681..682..683..684..685..686..687..688..689..690..691..692..693..694..695..696..697..698..699..700..701..702..703..704..705..706..707..708..709..710..711..712..713..714..715..716..717..718..719..720..721..722..723..724..725..726..727..728..729..730..731..732..733..734..735..736..737..738..739..740..741..742..743..744..745..746..747..748..749..750..751..752..753..754..755..756..757..758..759..760..761..762..763..764..765..766..767..768..769..770..771..772..773..done in 2177.9720 seconds
60000
0..1..2..3..4..5..6..7..8..9..10..11..12..13..14..15..16..17..18..19..20..21..22..23..24..25..26..27..28..29..30..31..32..33..34..35..36..37..38..39..40..41..42..43..44..45..46..